In [71]:
import numpy as np
import matplotlib
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
import copy

In [72]:
class PlayBig_17:
  def __init__(self,key):
    self.mat='' 
    self.a=[]
    self.key=key
    
  def preprocessing(self,plaintext):
    for i in range(len(plaintext)-2):
      if plaintext[i]==plaintext[i+1]:
        plaintext=plaintext[:i+1]+'x'+plaintext[i+1:]
    if len(plaintext)%2==1:
      plaintext+='x'
    return plaintext
  
  def postprocessing(self,plaintext):
    if plaintext[-1]=='x':
      plaintext = plaintext[:-1]   
    for i in range(1,len(plaintext)-3):
      if plaintext[i]=='x':
        if plaintext[i-1]==plaintext[i+1]:    
          plaintext=plaintext[:i]+plaintext[i+1:]
    return plaintext
        
  def createMatrix(self):
    key1=set(self.key)
    tmp='abcdefghijklmnopqrstuvwxyz0123456789 +-*/%()?=,\t'
    set1=set(tmp)
    set2=set1-key1
    mat=''.join(key1)+''.join(set2)
  
    a=[]
    # error
    for i in range(7):
      tmp=[]
      for j in range(7):
        tmp.append(mat[(7*i)+j])
      a.append(tmp)
    self.a=a
    self.mat=mat

  
  def locate(self,b,c):
    b,c=self.mat.index(b),self.mat.index(c)
    b=[b//7,b%7]
    c=[c//7,c%7]
    return b,c
  
  def substituteE(self,b,c): 
    a=self.a 
    if b[0]==c[0]:
        d=a[b[0]][(b[1]+1)%7]
        e=a[c[0]][(c[1]+1)%7]
    if b[1]==c[1]:
        d=a[(b[0]+1)%7][b[1]]
        e=a[(c[0]+1)%7][c[1]]
    if b[1]!=c[1] and b[0]!=c[0]:
        d=a[b[0]][c[1]]
        e=a[c[0]][b[1]]
    return d,e

  def substituteD(self,b,c):
    d=''
    e=''
    a=self.a
    if b[0]==c[0]:
        d=a[b[0]][(b[1]-1)%7]
        e=a[c[0]][(c[1]-1)%7]
    if b[1]==c[1]:
        d=a[(b[0]-1)%7][b[1]]
        e=a[(c[0]-1)%7][c[1]]
    if b[1]!=c[1] and b[0]!=c[0]:
        d=a[b[0]][c[1]]
        e=a[c[0]][b[1]]
    return d,e

  def encrypt(self,plaintext):
    
      ciphertext=''
      for i in range(0,len(plaintext),2):
          b,c=self.locate(plaintext[i],plaintext[i+1])
          d,e=self.substituteE(b,c)
          ciphertext+=d+e
      return ciphertext
  def decrpyt(self,ciphertext):
      plaintext=''
      for i in range(0,len(ciphertext),2):
          b,c=self.locate(ciphertext[i],ciphertext[i+1])
          d,e=self.substituteD(b,c)
          plaintext+=d+e
      return plaintext

     
  def stagno_img(self,img,binData):
    flag=0
    img1=copy.deepcopy(img)
    len1=len(img)
    len2=len(img[0])
    len3=len(img[0][0])
  
    x=0
    for i in range(len1):
      for j in range(len2):
        for k in range(len3):
        
          if x>=len(binData):
            img1[i][j][k]+=10
            flag=1
            break
          if binData[x]=='1': img1[i][j][k]-=1
          x+=1
        if flag==1:
          break  
      if flag==1:
        break

    # Output Images
    # plt.imshow(img1)
    # plt.show()
    mpimg.imsave('encrypted.bmp',format='bmp',arr= img1)
    mpimg.imsave('Original.bmp',format='bmp',arr= img)
    # matplotlib.image.imsave('encrypted.jpg', img1)
    return img1
    # plt.imshow(img)
    # plt.show()
    
  def stagno(self,img_path,encrypt):
    data=encrypt
    length=len(data)
 
    binData=''.join(format(ord(i), '08b') for i in data)
    img = mpimg.imread(img_path,format='jpg')
    # plt.imshow(img)
    img = np.asarray(img)  
    return img,self.stagno_img(img,binData)
  
  
  def stagno_decrypt(self,img1,img):
    len1=len(img)
    len2=len(img[0])
    len3=len(img[0][0])

    flag=0
    decrypted=''
    for i in range(len1):
      for j in range(len2):
        for k in range(len3):
          if img1[i][j][k]-img[i][j][k]==10:
            flag=1
            break
          val=img[i][j][k]-img1[i][j][k]
         
          decrypted+=str(val)
        if flag==1:break
      if flag==1:
        break

    
    binary_int = int(decrypted, 2)
    byte_number = binary_int.bit_length() + 7 // 8
    binary_array = binary_int.to_bytes(byte_number, "big")
    ascii_text = binary_array.decode()
    return ascii_text.split('\x00')[-1] 
    

In [73]:
plaintext = open('data.txt','r').read()
plaintext =plaintext.replace('\n',' ')

In [74]:
obj = PlayBig_16('geeks')

# plaintext = open('test.txt','r').read()
# plaintext = "Hello saf afs sabf fsa sa"
# plaintext= file
plaintext=plaintext.lower()

plaintext = obj.preprocessing(plaintext=plaintext)
print("Processed Plaintext ",plaintext)
obj.createMatrix()
cipher = obj.encrypt(plaintext=plaintext)
print("\nPlay fair cipher",cipher)


Processed Plaintext   x helxlo wor3ld techini2cal gexeks1 hix

Play fair cipher ewky1drdnk2nxpylkjk8loonco7yksd81ge kyzr


In [75]:
len('abcdefghijklmnopqrstuvwxyz0123456789 +-*/%()?=,\n\t')

49

In [76]:
import random
# print("decrypted play fair",obj.decrpyt(ciphertext=cipher))
path='./images/img{}.jpg'.format(random.randint(1,7))
img , img1 = obj.stagno(img_path=path,encrypt=cipher)
img1 = np.asarray(mpimg.imread('encrypted.bmp'))
cipher = obj.stagno_decrypt(img=img,img1=img1)
print("Stagno decrypted cipher ",cipher)
plaintext =obj.decrpyt(ciphertext=cipher)
print("Finally decrypted cipher : ",obj.postprocessing(plaintext=plaintext))




Stagno decrypted cipher  ewky1drdnk2nxpylkjk8loonco7yksd81ge kyzr
Finally decrypted cipher :    hello wor3ld techini2cal geeks1 hi


C:\Users\Rushabh\AppData\Local\Temp\ipykernel_10792\2122469097.py:147: RuntimeWarning: overflow encountered in ubyte_scalars
  if img1[i][j][k]-img[i][j][k]==10:
